In [0]:
import re
import pickle
import os
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob import classifiers
import nltk
from difflib import SequenceMatcher

def similar(a,b):
  return SequenceMatcher(None, a, b).ratio()

In [19]:
from google.colab import drive
drive.mount('/content/drive')
file_cl = open("/content/drive/My Drive/Colab Notebooks/naive_bayes.obj","rb")
classifier = pickle.load(file_cl)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
temp = [{'A':'dummy', 'B':'dummy'}]
mapping = pd.DataFrame(temp)  
index=1
#print(mapping)

In [0]:
def preprocess(tweet):
    tweet = re.sub(r'@[\w]*|#|[^\w?.!\s]|\bhttps?[^\s]*\b', '', tweet)
    return tweet
  
def predict(tweet):
  result = classifier.classify(tweet)
  prob_dist=classifier.prob_classify(tweet)
  print("Predicted: " + result + ", Positive: " + str(round(prob_dist.prob("pos"), 2)) + ", Negative: " + str(round(prob_dist.prob("neg"), 2)))
  posprob = round(prob_dist.prob("pos"), 2)
  negprob = round(prob_dist.prob("neg"),2)
  
  if (result == "neg"):
    return 0, posprob, negprob
  elif (result == "pos"):
    return 1, posprob, negprob
      
def userFeature(tweet, posprob, negprob):
  global mapping
 
  if (posprob < 0.90 and posprob > 0.10):
        for index, row in mapping.iterrows():
          if (similar(row['A'], tweet)>0.6):
            tweet = tweet + row['B']
            return predict(tweet)
          
        print("please help out boi: " + tweet)
        
        reformulation = input()
        ##print(reformulation)
        toAppend = pd.DataFrame({'A': [tweet], 'B': [reformulation]})
        #print("yeet")
        #print(toAppend)
        mapping = mapping.append(toAppend, ignore_index= True)
        
        tweet = tweet + reformulation
        
        res2, posprob2, negprob2 = predict(tweet)
        
        return res2, posprob2, negprob2


In [145]:
test_phrases = {
    "#inaperfectworld final exams wouldn't exist and i would be sleeping very confy right now!": 1,
    "ok thats it you win.": 0,
    "#Swim&amp;Run #Cologne was great!Lesson learnedon't bring water w/ gas.Cost me 5-7min.,easy ": 1,
    "#Tetris, one of the most renowned and addictive creations in the brief history of video games, turns 25 this week  Long live Tetris!": 1,
    "#TFARP -cycle. Been a rather long solar cycle for everybody, I think. :p G'night Bots, Cons, and assorted others! ": 0,
    "#thankyouGod for all the mothers in the world who doesnt know how to cook salsa but will be willing to try for their children ": 1,
    "#inaperfectworld i'd be sitting VIP row 1 at the game tonight!": 1,
    "#iPhone - updated to 3.0 but cant activate because  iTunes store activation connection time out error ": 0,
    "#Up is amazing!  I'm glad I got to see it &amp; see @KristinRose84 too!": 1,
    "#Music happy music monday everybody, I LOVE MUSIC ": 1
}	

secret_test_phrases = {
}


if __name__ == "__main__":
    
    nbr_test_phrases = len(test_phrases)
    nbr_secret_test_phrases = len(secret_test_phrases)

    count_correct = 0
    
    for phrase, score in test_phrases.items():
        formatted = preprocess(phrase)
        res, posprob, negprob = predict(formatted)
        if (posprob > 0.90 or posprob < 0.10):
          if res == score:
              count_correct += 1
        else:
          res2, posprob2, negprob2 = userFeature(formatted, posprob, negprob)
          if res2 == score:
            count_correct += 1

    test_phases_count_correct = count_correct
    test_phrases_result = test_phases_count_correct/nbr_test_phrases * 100
    print('You have a score of {}% on the test phrases'.format(test_phrases_result))
    
    if test_phrases_result < 70:
    	print("You do not get a good score on the test phrases! {}".format("*" * 30))

    count_correct = 0
    for phrase, score in secret_test_phrases.items():
        res = sentiment_analysis_result(phrase)
        if res == score:
            count_correct += 1

    secret_test_phases_count_correct = count_correct
    secret_test_phrases_result = 0.0
    if nbr_secret_test_phrases != 0:
    	secret_test_phrases_result = secret_test_phases_count_correct/nbr_secret_test_phrases * 100
    print('You have a score of {}% on the secret test phrases'.format(secret_test_phrases_result))
    
    total_test_phrases_result = ((test_phases_count_correct + secret_test_phases_count_correct)/ \
    	(nbr_test_phrases + nbr_secret_test_phrases)) * 100

    print('You have a total score of {}%'.format(total_test_phrases_result))


Predicted: pos, Positive: 0.68, Negative: 0.32
please help out boi: inaperfectworld final exams wouldnt exist and i would be sleeping very confy right now!
in a perfect world, final exams would not exist and i would be sleeping super comfy right now!
Predicted: pos, Positive: 0.56, Negative: 0.44
Predicted: pos, Positive: 0.74, Negative: 0.26
please help out boi: ok thats it you win.
i lost
Predicted: neg, Positive: 0.3, Negative: 0.7
Predicted: pos, Positive: 0.9, Negative: 0.1


TypeError: ignored

In [132]:
print(mapping)

                                                   A  \
0                                              dummy   
0  inaperfectworld final exams wouldnt exist and ...   
0                               ok thats it you win.   

                                                B  
0                                           dummy  
0  in a perfect world final exams would not exist  
0                                          i lost  
